### Matrix Shape Tests for Applying Gains Matrix

In [1]:
import cupy as cp
from utils import *
from zp_puregpu_funcs_py import *

In [2]:
apply_gains_to_mat?

Signature:
apply_gains_to_mat(
    gains,
    mat,
    edges,
    ant_1_array,
    ant_2_array,
    xp,
    is_zeropadded=True,
)
Docstring:
Apply a pair of complex gains to a matrix. Utilizes the Re/Im split.
Only accounts for "one half" of the gain application, meaning the 
function is really performing (g_1g_2*\Delta_{1,2}), where it is 
understood that antenna's 1 and 2 below to the baseline sitting at
the same row as that baseline row in the \Delta matrix.

Params
------
mat: Gains are applied to this. Can be 2d as in original C-corrcal.
    If 3d, is_zeropadded must be set to True

Returns
-------
out: Matrix with applied gains (explain this a bit better)
File:      ~/corrcal_gpu_pipeline/pipeline/utils.py
Type:      function

In [3]:
#simulation params
n_ant = 10
n_bl = 2*n_ant
n_gains = 4*n_ant
n_eig = 3
n_src = 5
xp = cp  #run things on the gpu using cupy

#random array of edges for the diffuse matrix
edges = xp.unique(xp.random.randint(1, n_bl-1, size = 2))
edges = xp.concatenate((xp.array([0]), edges, xp.array([n_bl])))
print(f"The edges of the redundant blocks have indices{edges}")

#some random noise, diffuse, and source covariance matrices
xp = cp
sim_noise_mat = xp.random.rand(n_bl, dtype = 'float64')**2   #in principle this is squared since is a variance
sim_diff_mat = xp.random.rand(n_bl, n_eig, dtype = 'float64')
sim_src_mat = xp.random.rand(n_bl, n_src, dtype = 'float64')
sim_data_vec = xp.random.rand(n_bl, dtype = 'float64')
sim_gains = cp.random.rand(n_gains, dtype = 'float64') #Re/Im split + ant1 & ant 2 = 4*n_ant

print(type(sim_noise_mat))
print(type(sim_diff_mat))
print(type(sim_src_mat))
print(type(sim_data_vec))

#zeropad the noise, diff, source mats
zp_sim_noise_mat_inv, largest_block, n_blocks = zeroPad(sim_noise_mat, edges, return_inv=True)  #return inverse set to True to get around div by 0 issue
zp_sim_noise_mat, largest_block, n_blocks = zeroPad(sim_noise_mat, edges, return_inv=False)  #return inverse set to False for sims later on
zp_sim_diff_mat, largest_block, n_blocks = zeroPad(sim_diff_mat, edges, return_inv=False)
zp_sim_src_mat, largest_block, n_blocks = zeroPad(sim_src_mat, edges, return_inv=False)

#Need to reshape to give an extra dimension of n_blocks to be compatible with inv cov routine
sim_noise_mat_2d = zp_sim_noise_mat_inv.reshape(n_blocks, largest_block)   #Again, its worth noting the values of the noise mat are really 1/noise_vals here
sim_diff_mat_3d = zp_sim_diff_mat.reshape(n_blocks, largest_block, n_eig)
sim_src_mat_3d = zp_sim_src_mat.reshape(n_blocks, largest_block, n_src)

The edges of the redundant blocks have indices[ 0  2 14 20]
<class 'cupy.ndarray'>
<class 'cupy.ndarray'>
<class 'cupy.ndarray'>
<class 'cupy.ndarray'>


In [4]:
import numpy as np
m = np.random.rand(5,6,3)
n = np.random.rand(5,6,3)
n = n.reshape(5*6, 3)
# b = m[:, ::2] * n[:, ::2]
# v = m * n
# print(b)
# print(v)
# print(m[:, ::2])

In [5]:
from utils import *


In [6]:
print(sim_diff_mat.shape)
print(zp_sim_diff_mat.shape)

(20, 3)
(36, 3)


In [7]:
ant_1_array = cp.arange(n_ant)
print(ant_1_array.shape)
print(sim_diff_mat.shape)
print(zp_sim_diff_mat.shape)
print(sim_diff_mat_3d.shape)
print(sim_gains.shape)

(10,)
(20, 3)
(36, 3)
(3, 12, 3)
(40,)


In [8]:
apply_gains_to_mat(sim_gains, sim_diff_mat_3d, edges, ant_1_array, ant_1_array, cp, True)


(3, 12, 3)

[0.63772347 0.98611188 0.9693958  0.08645951 0.72853302 0.91334308
 0.62860848 0.73099463 0.51909912 0.16445776 0.63789684 0.64339438
 0.94515611 0.70859123 0.33437534 0.82088593 0.0935104  0.75720455
 0.9136245  0.03193133]
[0.63772347 0.98611188 0.9693958  0.08645951 0.72853302 0.91334308
 0.62860848 0.73099463 0.51909912 0.16445776 0.63772347 0.98611188
 0.9693958  0.08645951 0.72853302 0.91334308 0.62860848 0.73099463
 0.51909912 0.16445776]
(36,)
[0.63772347 0.98611188 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.9693958  0.08645951 0.72853302 0.91334308 0.62860848 0.73099463
 0.51909912 0.16445776 0.63772347 0.98611188 0.9693958  0.08645951
 0.72853302 0.91334308 0.62860848 0.73099463 0.51909912 0.16445776
 0.         0.         0.         0.         0.         0.        ]
(10,)
(10,)
(10,)
(18,)
(18,)
(18,)
[[0.66208339+8.89870062e-18j]
 [0.3243475 +2.48803399e-19j]
 [0.        +0.00000000e+00j]
 [0.        +0.0000

ValueError: cannot reshape array of size 18 into shape (3, 12, 1)

In [13]:
m = np.random.rand(2, 5, 3)
g = np.random.rand(10)

gs = g[:, None] * g[:, None]
gs = gs.reshape(m.shape[0], m.shape[1], 1)

print(gs.shape)
print(m.shape)
print(g.shape)
print(m)
print(m[:, ::2])
out = gs[:, ::2] * m[:, ::2]

(2, 5, 1)
(2, 5, 3)
(10,)
[[[0.28792675 0.23025634 0.6019926 ]
  [0.3649799  0.83724445 0.97892857]
  [0.8108697  0.30838806 0.86713359]
  [0.67252363 0.45473794 0.15374954]
  [0.44281979 0.20866379 0.31340939]]

 [[0.24300186 0.17208641 0.08687405]
  [0.38217043 0.44386447 0.22144284]
  [0.86703401 0.79247312 0.22997477]
  [0.71024074 0.63128673 0.67401253]
  [0.09133763 0.85180797 0.44132384]]]
[[[0.28792675 0.23025634 0.6019926 ]
  [0.8108697  0.30838806 0.86713359]
  [0.44281979 0.20866379 0.31340939]]

 [[0.24300186 0.17208641 0.08687405]
  [0.86703401 0.79247312 0.22997477]
  [0.09133763 0.85180797 0.44132384]]]


In [15]:
a = np.arange(5)
b = np.arange(5,10)
c = np.random.rand((a.shape[0] + b.shape[0]))
print(c)
d = c[a]
e = c[b]

print(d)
print()
print(e)
print()
print(np.concatenate((d, e)))
print()
aa = np.concatenate((d, e))

print(aa[:int(len(aa)/2)])

print(aa[int(len(aa)/2):])

[0.7168376  0.09178836 0.30719332 0.64421814 0.7537436  0.44306968
 0.13087968 0.4341032  0.62679891 0.60623726]
[0.7168376  0.09178836 0.30719332 0.64421814 0.7537436 ]

[0.44306968 0.13087968 0.4341032  0.62679891 0.60623726]

[0.7168376  0.09178836 0.30719332 0.64421814 0.7537436  0.44306968
 0.13087968 0.4341032  0.62679891 0.60623726]

[0.7168376  0.09178836 0.30719332 0.64421814 0.7537436 ]
[0.44306968 0.13087968 0.4341032  0.62679891 0.60623726]
